In [1]:
import pandas as pd
import numpy as np
import copy
import math

# Importação dos csv

In [3]:
autores = pd.read_csv('./datasets/authors.csv')
categorias = pd.read_csv('./datasets/categories.csv')
formatos = pd.read_csv('./datasets/formats.csv')
ds = pd.read_csv('./datasets/dataset.csv')

aux = {'authors': autores,
       'categories': categorias,
       'format': formatos}

# Definição das Funções

In [4]:
# Função para transformar em listas as strings que representam listas dentro do dataset
def clean_alt_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_

In [5]:
# Função para cruzar os dados e gerar um dataset unificado
def join_dfs(df_main_id_column, df_aux, df_main):
    
    # Obtém nome da coluna de identificação e o nome da coluna de descrição
    # das tabelas auxiliares à dataset
    id_column = df_aux.columns[0]
    name_column = df_aux.columns[1]
    
    # Gera um dicionário a ser utilizado para unir as informações dos dfs
    dict_aux = df_aux.set_index(id_column)[name_column].to_dict()
    
    if name_column not in df_main.columns:   
        
        # Localiza a posição da coluna do dataset referente ao id da auxiliar
        # e cria uma nova coluna na posição seguinte
        ix = df_main.columns.get_loc(df_main_id_column) + 1
        df.insert(ix, name_column, '')
        
        # Se a coluna que referencia a tabela auxiliar é uma string que representa lista
        # ela será tratada para virar uma lista
        if df[df_main_id_column].dtype == 'object':
            df[df_main_id_column] = df[df_main_id_column].apply(clean_alt_list)
            df[df_main_id_column] = df[df_main_id_column].apply(eval)
    
    # Se a coluna que referencia a tabela auxiliar é uma lista, as descrições 
    # serão unidas em uma string e adicionadas a nova coluna
    if df[df_main_id_column].dtype == 'object':
        df[name_column] = ['; '.join([str(dict_aux[int(j)]) for j in i if j != '']) for i in df[df_main_id_column]]
    
    # Caso contrátio os valores serão inseridos diretamente
    else:
        df[name_column] = [(dict_aux[i] if not math.isnan(i) else '') for i in df[df_main_id_column]]

In [6]:
# Função para transformar as colunas com listas em colunas com valores únicos (1D)
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

# Join information

In [7]:
# Cria-se uma cópia da dataset e une as informações das tabelas auxiliares a ela
df = copy.deepcopy(ds)
for aux_id_name, aux_df in aux.items():
    join_dfs(aux_id_name, aux_df, df)

# Questões

## Transformações

In [8]:
# Função para agrupar os df e ordená-los a partir da sua contagem
def group_and_sort(id_name, df_aux, df_main):
    id_column = df_aux.columns[0]
    name_column = df_aux.columns[1]

    df_group = pd.DataFrame({id_column: to_1D(df_main[id_name]), 'contagem': 1})
    df_group[id_column] = pd.to_numeric(df_group[id_column])
    df_group = pd.merge(df_group, df_aux)[[name_column, 'contagem']]
    df_group = df_group.groupby(name_column, as_index=False).count()
    df_group = df_group.sort_values(name_column)
    
    df_sort = df_group.sort_values('contagem', ascending=False)
    return df_group, df_sort

In [9]:
# Transformações Questão 2
df.insert(2, 'author_quantity', 0)
df['author_quantity'] = df.apply(lambda row: len(row['authors']), axis=1)

In [10]:
# Transformações Questão 3
_, ranked_authors = group_and_sort('authors', autores, df)
ranked_authors = ranked_authors.head()

In [11]:
# Transformações Questão 4 e Questão 5
total_category, ranked_category = group_and_sort('categories', categorias, df)
ranked_category = ranked_category.head()

In [12]:
# Transformações Questão 6:
_ = df[['format_name', 'title']].groupby('format_name', as_index=False).count().sort_values('title', ascending=False)
most_used_format, quantity_format = _.head(1).iloc[0]

In [13]:
# Transformações Questão 7:
ranked_bestseller = df[['title', 'bestsellers-rank']].sort_values('bestsellers-rank').head(10)

In [14]:
# Transformações Questão 8:
ranked_rating = df[['title', 'rating-avg']].sort_values('rating-avg', ascending=False).head(10)

In [15]:
# Transformações Questão 9:
best_ratings = df[df['rating-avg'] > 3.5].title.count()

In [16]:
# Transformações Questão 10:
books_new_decade = df[df['publication-date'] > '2020-01-01'].title.count()

## Soluções

In [17]:
# Questão 1:
print('Há {} livros no dataset'.format(df.title.count()))

Há 20216 livros no dataset


In [18]:
# Questão 2:
print('Há {} livros com apenas 1 autor'.format(df[df['author_quantity'] == 1].title.count()))

Há 15341 livros com apenas 1 autor


In [19]:
# Questão 3:
print('Top 5 autores com maiores quantidades de livros:\n')
for ix, row in enumerate(ranked_authors.values):
    print('{}° autor: {} com {} livros'.format(ix+1, row[0], row[1]))
    

Top 5 autores com maiores quantidades de livros:

1° autor: Weezag com 258 livros
2° autor: Rand McNally com 212 livros
3° autor: William Shakespeare com 117 livros
4° autor: Matthew Preston com 83 livros
5° autor: Kevin Lee com 83 livros


In [20]:
# Questão 4:
print('Quantidade de livros por categoria:\n')
for row in total_category.values:
    print('Categoria: {:75} Livros: {}'.format(row[0], row[1]))

Quantidade de livros por categoria:

Categoria: 20th Century & Contemporary Classical Music                                 Livros: 4
Categoria: 20th Century History: C 1900  To C 2000                                     Livros: 92
Categoria: 21st Century History: From C 2000 -                                         Livros: 19
Categoria: ABC                                                                         Livros: 2
Categoria: Abnormal Psychology                                                         Livros: 17
Categoria: Accident & Emergency Medicine                                               Livros: 1
Categoria: Accounting                                                                  Livros: 4
Categoria: Acoustic & Sound Engineering                                                Livros: 9
Categoria: Acquisitions & Collection Development                                       Livros: 2
Categoria: Acting Techniques                                                           

In [21]:
# Questão 5:
print('Top 5 categorias com maiores quantidades de livros:\n')
for ix, row in enumerate(ranked_category.values):
    print('{}° categoria: {} com {} livros'.format(ix+1, row[0], row[1]))
    

Top 5 categorias com maiores quantidades de livros:

1° categoria: Television com 4450 livros
2° categoria: Teaching Skills & Techniques com 4122 livros
3° categoria: World Atlases / World Maps com 3792 livros
4° categoria: Usage & Grammar Guides com 1862 livros
5° categoria: Theatre Studies com 1854 livros


In [22]:
# Questão 6:
print('Formato mais utilizado: {} com {} utilizações'.format(most_used_format, quantity_format))

Formato mais utilizado: Paperback com 13497 utilizações


In [23]:
# Questão 7:
print('Livros mais bem posicionados relativos a bestseller:\n')
for row in ranked_bestseller.values:
    print('Livro: "{}" na posição {}'.format(row[0], row[1]))

Livros mais bem posicionados relativos a bestseller:

Livro: "We" na posição 2038.0
Livro: "Mushrooms" na posição 5718.0
Livro: "Enlightenment Now : The Case for Reason, Science, Humanism, and Progress" na posição 11401.0
Livro: "Affective Dimensions in Chemistry Education" na posição 12512.0
Livro: "The Science Delusion : Freeing the Spirit of Enquiry (NEW EDITION)" na posição 14602.0
Livro: "The Age of Surveillance Capitalism : The Fight for a Human Future at the New Frontier of Power: Barack Obama's Books of 2019" na posição 14656.0
Livro: "Quirkology : The Curious Science Of Everyday Lives" na posição 14804.0
Livro: "Quiet : The power of introverts in a world that can't stop talking" na posição 14986.0
Livro: "The Brain's Way of Healing : Stories of Remarkable Recoveries and Discoveries" na posição 15844.0
Livro: "Enlightenment Now" na posição 16270.0


In [24]:
# Questão 8:
print('Livros mais bem posicionados relativos a média de nota:\n')
for row in ranked_rating.values:
    print('Livro: "{}" com nota {}'.format(row[0], row[1]))

Livros mais bem posicionados relativos a média de nota:

Livro: "The Oxford Handbook of Early Christian Archaeology" com nota 5.0
Livro: "Edward Albee : A Research and Production Sourcebook" com nota 5.0
Livro: "Asia Classic, Tubed : Wall Maps Continents" com nota 5.0
Livro: "Hauntingly Familiar : A Paranormal Thriller" com nota 5.0
Livro: "The Other Judaisms of Late Antiquity : Second Edition" com nota 5.0
Livro: "Freaky Florida : The Wonderhouse, the Devil's Tree, the Shaman of Philippe Park, and More" com nota 5.0
Livro: "Families, Rabbis, and Education : Traditional Jewish Society in Nineteenth-century Eastern Europe" com nota 5.0
Livro: "Total Transformation : Seven Sacred Steps to Freedom" com nota 5.0
Livro: "Strategies for the Tech-Savvy Classroom" com nota 5.0
Livro: "Spirals of Love : A Journey Into the Abyss of the Heartspace" com nota 5.0


In [25]:
# Questão 9:
print('{} livros possuem nota maior que 3,5'.format(best_ratings))

8182 livros possuem nota maior que 3,5


In [26]:
# Questão 10:
print('{} livros foram publicados após 01-01-2020'.format(books_new_decade))

1178 livros foram publicados após 01-01-2020


# Criação dos arquivos de saída

In [29]:
# Arquivo dataset com junções
df.to_csv('./resolucao/dataset_unificado.csv', index=False)

# Arquivo questão 4
total_category.to_csv('./resolucao/questao_4.csv', index=False, sep=';')

# Arquivo questões 1 a 3 e 5 a 10
with open('./resolucao/solucoes.txt', 'w') as outfile:
    outfile.write('>> Questão 1:\n')
    outfile.write('Há {} livros no dataset\n\n'.format(df.title.count()))
    
    outfile.write('>> Questão 2:\n')
    outfile.write('Há {} livros com apenas 1 autor\n\n'.format(df[df['author_quantity'] == 1].title.count()))
    
    outfile.write('>> Questão 3:\n')
    outfile.write('Top 5 autores com maiores quantidades de livros:\n')
    for ix, row in enumerate(ranked_authors.values):
        outfile.write('{}° autor: "{}" com {} livros\n'.format(ix+1, row[0], row[1]))
    
    
    outfile.write('\n>> Questão 5:\n')
    outfile.write('Top 5 categorias com maiores quantidades de livros:\n')
    for ix, row in enumerate(ranked_category.values):
        outfile.write('{}° categoria: "{}" com {} livros\n'.format(ix+1, row[0], row[1]))
    
    outfile.write('\n>> Questão 6:\n')
    outfile.write('Formato mais utilizado: "{}" com {} utilizações\n\n'.format(most_used_format, quantity_format))
    
    outfile.write('>> Questão 7:\n')
    outfile.write('Livros mais bem posicionados relativos a bestseller:\n')
    for row in ranked_bestseller.values:
        outfile.write('Livro: "{}" na posição {}\n'.format(row[0], row[1]))
        
    outfile.write('\n>> Questão 8:\n')
    outfile.write('Livros mais bem posicionados relativos à média de nota:\n')
    for row in ranked_rating.values:
        outfile.write('Livro: "{}" com nota {}\n'.format(row[0], row[1]))
    
    
    outfile.write('\n>> Questão 9:\n')
    outfile.write('{} livros possuem nota maior que 3,5\n\n'.format(best_ratings))
    
    outfile.write('>> Questão 10:\n')
    outfile.write('{} livros foram publicados após "01-01-2020"'.format(books_new_decade))